In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("btc_price_prediction") \
    .config(
        "spark.jars.packages",
        "org.postgresql:postgresql:42.7.3"
    ) \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")



your 131072x1 screen size is bogus. expect trouble
26/01/23 10:29:59 WARN Utils: Your hostname, DESKTOP-9V99V1S resolves to a loopback address: 127.0.1.1; using 172.20.61.196 instead (on interface eth0)
26/01/23 10:29:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/mnt/c/Users/khadija/Desktop/BTC-Prediction/quantai-btc-price-prediction-platform/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/khadija/.ivy2/cache
The jars for the packages stored in: /home/khadija/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0d2b394a-9374-4729-9bc7-493513126d48;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 544ms :: artifacts dl 29ms
	:: modules in use:
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------------

In [2]:
#spark.stop()

## 1. Data Injection (collecte data)

In [3]:
import requests
import os
from dotenv import load_dotenv
load_dotenv() 
def data_collection():

  SYMBOL = 'BTCUSDT'
  INTERVAL = '1m'   # Intervalle d'une minute
  LIMIT = 3000   
  api='https://api.binance.com/api/v3/klines'
  response = requests.get(api, params={
    "symbol": SYMBOL,
    "interval": INTERVAL,
    "limit": LIMIT
  })
  if response.status_code != 200:
        print(f"Erreur {response.status_code}")
        return None
  api_data=response.json()
  columns = [
        "open_time", "open_price", "high_price", "low_price", "close_price", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_volume", "taker_buy_quote_volume", "ignore"
    ]
  psdf= spark.createDataFrame(api_data, columns)
 
  
  return psdf
  
data = data_collection()
data.show(10)

+-------------+--------------+--------------+--------------+--------------+-----------+-------------+------------------+----------------+---------------------+----------------------+------+
|    open_time|    open_price|    high_price|     low_price|   close_price|     volume|   close_time|quote_asset_volume|number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|
+-------------+--------------+--------------+--------------+--------------+-----------+-------------+------------------+----------------+---------------------+----------------------+------+
|1769100660000|89089.58000000|89120.00000000|89062.68000000|89119.99000000| 4.19366000|1769100719999|   373564.95759390|            3603|           1.35785000|       120974.39408560|     0|
|1769100720000|89119.99000000|89146.90000000|89087.31000000|89087.31000000| 5.69300000|1769100779999|   507414.36134390|            3952|           2.46532000|       219752.29222980|     0|
|1769100780000|89087.31000000|89087.32000000|88953

In [4]:
data.printSchema()

root
 |-- open_time: long (nullable = true)
 |-- open_price: string (nullable = true)
 |-- high_price: string (nullable = true)
 |-- low_price: string (nullable = true)
 |-- close_price: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- close_time: long (nullable = true)
 |-- quote_asset_volume: string (nullable = true)
 |-- number_of_trades: long (nullable = true)
 |-- taker_buy_base_volume: string (nullable = true)
 |-- taker_buy_quote_volume: string (nullable = true)
 |-- ignore: string (nullable = true)



In [5]:
print(f"nombre des lignes dana la dataset :",data.count())

nombre des lignes dana la dataset : 1000


In [6]:
from pyspark.sql.functions import col
num_cols= ["open_price", "high_price", "low_price", "close_price", "volume", "quote_asset_volume", "number_of_trades", "taker_buy_base_volume", "taker_buy_quote_volume"]
for feature in num_cols:
      data = data.withColumn(feature, col(feature).cast('double'))

data.printSchema()

root
 |-- open_time: long (nullable = true)
 |-- open_price: double (nullable = true)
 |-- high_price: double (nullable = true)
 |-- low_price: double (nullable = true)
 |-- close_price: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- close_time: long (nullable = true)
 |-- quote_asset_volume: double (nullable = true)
 |-- number_of_trades: double (nullable = true)
 |-- taker_buy_base_volume: double (nullable = true)
 |-- taker_buy_quote_volume: double (nullable = true)
 |-- ignore: string (nullable = true)



In [7]:
data.describe().show()

+-------+--------------------+------------------+------------------+------------------+------------------+----------------+--------------------+------------------+------------------+---------------------+----------------------+------+
|summary|           open_time|        open_price|        high_price|         low_price|       close_price|          volume|          close_time|quote_asset_volume|  number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|
+-------+--------------------+------------------+------------------+------------------+------------------+----------------+--------------------+------------------+------------------+---------------------+----------------------+------+
|  count|                1000|              1000|              1000|              1000|              1000|            1000|                1000|              1000|              1000|                 1000|                  1000|  1000|
|   mean|       1.76913063E12| 89581.12319000001| 89601.7670

1. Convertir les colonnes open_time (ms) et close_time (ms) en secondes (s).

In [8]:


data= data.withColumn("open_time_s", col("open_time") / 1000) \
       .withColumn("close_time_s", col("close_time") / 1000)

data.columns
data.select("open_time_s","open_time").show(5)

+------------+-------------+
| open_time_s|    open_time|
+------------+-------------+
|1.76910066E9|1769100660000|
|1.76910072E9|1769100720000|
|1.76910078E9|1769100780000|
|1.76910084E9|1769100840000|
| 1.7691009E9|1769100900000|
+------------+-------------+
only showing top 5 rows



2. Convertir les colonnes open_time_s et close_time_s  en timestamp.

In [9]:
from pyspark.sql.functions import to_timestamp, col

data = data.withColumn("open_time_ts", to_timestamp(col("open_time_s"))) \
       .withColumn("close_time_ts", to_timestamp(col("close_time_s")))
data.printSchema()



root
 |-- open_time: long (nullable = true)
 |-- open_price: double (nullable = true)
 |-- high_price: double (nullable = true)
 |-- low_price: double (nullable = true)
 |-- close_price: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- close_time: long (nullable = true)
 |-- quote_asset_volume: double (nullable = true)
 |-- number_of_trades: double (nullable = true)
 |-- taker_buy_base_volume: double (nullable = true)
 |-- taker_buy_quote_volume: double (nullable = true)
 |-- ignore: string (nullable = true)
 |-- open_time_s: double (nullable = true)
 |-- close_time_s: double (nullable = true)
 |-- open_time_ts: timestamp (nullable = true)
 |-- close_time_ts: timestamp (nullable = true)



### create le target y = close_price(t+10)

In [10]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

# Creation une fenêtre ordonnée par temps
window = Window.orderBy("open_time_ts")

# Décaler la colonne 'close_price' de 10 lignes (10 minutes)
data_new = data.withColumn("close_t_plus_10", F.lead("close_price", 10).over(window))
data_new.select("close_price","close_t_plus_10","open_time_ts").show()

+-----------+---------------+-------------------+
|close_price|close_t_plus_10|       open_time_ts|
+-----------+---------------+-------------------+
|   89119.99|       89173.99|2026-01-22 17:51:00|
|   89087.31|       89156.51|2026-01-22 17:52:00|
|    88980.0|       89175.17|2026-01-22 17:53:00|
|   89002.22|       89175.99|2026-01-22 17:54:00|
|   88950.01|       89215.88|2026-01-22 17:55:00|
|   88926.23|        89184.4|2026-01-22 17:56:00|
|   89020.72|       89206.54|2026-01-22 17:57:00|
|   88983.67|       89321.49|2026-01-22 17:58:00|
|   89054.06|       89340.16|2026-01-22 17:59:00|
|    89026.0|       89407.81|2026-01-22 18:00:00|
|   89173.99|       89359.48|2026-01-22 18:01:00|
|   89156.51|        89236.0|2026-01-22 18:02:00|
|   89175.17|       89302.43|2026-01-22 18:03:00|
|   89175.99|       89228.33|2026-01-22 18:04:00|
|   89215.88|       89250.38|2026-01-22 18:05:00|
|    89184.4|       89320.91|2026-01-22 18:06:00|
|   89206.54|       89307.87|2026-01-22 18:07:00|


#### create colonne variation du prix (return) : return = close(t−1) / close(t) − close(t−1)

In [11]:
from pyspark.sql import functions as F
data_new = data_new.withColumn("precedent_close",F.lag("close_price",1).over(window))
data_new = data_new.withColumn('returns',(F.col("close_price")- F.col("precedent_close"))/F.col("precedent_close"))
data_new.select("close_price","precedent_close","returns").show(5)

+-----------+---------------+--------------------+
|close_price|precedent_close|             returns|
+-----------+---------------+--------------------+
|   89119.99|           NULL|                NULL|
|   89087.31|       89119.99|-3.66696630015415...|
|    88980.0|       89087.31|-0.00120454866130...|
|   89002.22|        88980.0|2.497190379860773...|
|   88950.01|       89002.22|-5.86614581074566...|
+-----------+---------------+--------------------+
only showing top 5 rows



#### Moyennes mobiles (5, 10 minutes)

In [12]:
window5t = window.orderBy("open_time_ts").rowsBetween(-5,-1)
window10t = window.orderBy("open_time_ts").rowsBetween(-10,-1)

data_new = data_new.withColumn("MA_5",F.avg("close_price").over(window5t))
data_new = data_new.withColumn("MA_10",F.avg("close_price").over(window10t))
data_new.select("close_price","open_time_ts","MA_5","MA_10").show(10)


+-----------+-------------------+-----------------+-----------------+
|close_price|       open_time_ts|             MA_5|            MA_10|
+-----------+-------------------+-----------------+-----------------+
|   89119.99|2026-01-22 17:51:00|             NULL|             NULL|
|   89087.31|2026-01-22 17:52:00|         89119.99|         89119.99|
|    88980.0|2026-01-22 17:53:00|         89103.65|         89103.65|
|   89002.22|2026-01-22 17:54:00|89062.43333333333|89062.43333333333|
|   88950.01|2026-01-22 17:55:00|         89047.38|         89047.38|
|   88926.23|2026-01-22 17:56:00|        89027.906|        89027.906|
|   89020.72|2026-01-22 17:57:00|88989.15400000001|         89010.96|
|   88983.67|2026-01-22 17:58:00|88975.83599999998|89012.35428571429|
|   89054.06|2026-01-22 17:59:00|88976.56999999998|      89008.76875|
|    89026.0|2026-01-22 18:00:00|        88986.938|89013.80111111111|
+-----------+-------------------+-----------------+-----------------+
only showing top 10 

#### Volume et Intensité de Trading (Taker Ratio)

In [13]:
data_new = data_new.withColumn("taker_ratio",F.col("taker_buy_base_volume")/F.col("volume"))
data_new.select("open_time_ts", "close_price", "returns",  "MA_5",  "MA_10",  "taker_ratio", "close_t_plus_10").show(10)



+-------------------+-----------+--------------------+-----------------+-----------------+-------------------+---------------+
|       open_time_ts|close_price|             returns|             MA_5|            MA_10|        taker_ratio|close_t_plus_10|
+-------------------+-----------+--------------------+-----------------+-----------------+-------------------+---------------+
|2026-01-22 17:51:00|   89119.99|                NULL|             NULL|             NULL| 0.3237863823009018|       89173.99|
|2026-01-22 17:52:00|   89087.31|-3.66696630015415...|         89119.99|         89119.99|0.43304408923239074|       89156.51|
|2026-01-22 17:53:00|    88980.0|-0.00120454866130...|         89103.65|         89103.65|0.11037938630946609|       89175.17|
|2026-01-22 17:54:00|   89002.22|2.497190379860773...|89062.43333333333|89062.43333333333|0.33019366973285924|       89175.99|
|2026-01-22 17:55:00|   88950.01|-5.86614581074566...|         89047.38|         89047.38| 0.6218183976337284| 

In [14]:
data_new = data_new.dropna(subset=["returns", "MA_5", "MA_10", "taker_ratio","close_t_plus_10"])
# print(data_new.count())
# data_new.printSchema()

In [15]:
silver_columns = [
        "open_time_ts", "open_price", "high_price", "low_price", "close_price", "volume",
        "returns", "MA_5", "MA_10", "taker_ratio", "close_t_plus_10"
    ]
df_silver = data_new.select(*silver_columns)

In [16]:
df_silver.columns

['open_time_ts',
 'open_price',
 'high_price',
 'low_price',
 'close_price',
 'volume',
 'returns',
 'MA_5',
 'MA_10',
 'taker_ratio',
 'close_t_plus_10']

In [35]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler ,StandardScaler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor


def training_evaluation_model(data):
 
    features_cols = ["returns", "MA_5", "MA_10", "taker_ratio"]
    
    target = "close_t_plus_10"

    assembler = VectorAssembler(inputCols=features_cols, outputCol="features",handleInvalid="skip")
    window = Window.orderBy("open_time_ts")
    df = data.withColumn("time_index", F.row_number().over(window))

   
    # Séparation séquentielle
    total_count = df.count()
    split_point = int(0.8 * total_count)


    
    train_data = df.filter(F.col("time_index") <= split_point)
    test_data= df.filter(F.col("time_index") > split_point)
    

    
    
    # Transormations
    
    scaler = StandardScaler(inputCol="features", outputCol="features_scaled", withStd=True, withMean=True)

    models = [
        ("Linear Regression", LinearRegression(featuresCol="features_scaled", labelCol=target)),
        ("Linear Regression ElasticNet", LinearRegression(featuresCol="features_scaled", labelCol=target, 
                                                           regParam=0.1, elasticNetParam=0.5)),
        
        ("Random Forest", RandomForestRegressor(featuresCol="features_scaled", labelCol=target, numTrees=100, maxDepth=10, 
                       seed=42)),
        
    ]  
    #  Évaluateurs
    evaluator_r2 = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="r2")
    evaluator_mae = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="mae")
    evaluator_rmse = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
    
    best_rmse = float("inf")
    best_model_name = None
    for model_name, model_algo in models:
        print(f"Entraînement du modèle : {model_name}...")
    
        # Créer pipeline
        pipeline = Pipeline(stages=[assembler, scaler, model_algo])

        # Entraîner modèle
        model_fit = pipeline.fit(train_data)
        # Prédiction
        preds = model_fit.transform(test_data)
        
        # Évaluation
        r2 = evaluator_r2.evaluate(preds)
        mae = evaluator_mae.evaluate(preds)
        rmse = evaluator_rmse.evaluate(preds)
        
        print(f"Modèle: {model_name}")

        print(f"R²: {r2:.4f} | MAE: {mae:.2f} | RMSE: {rmse:.2f}")
        
        if rmse < best_rmse:
            best_rmse = rmse
            best_model_fit = model_fit
            best_model_name = model_name


    if best_model_fit :
        save_path  = f"saved_model/{best_model_name}_pipeline"
        model_fit.write().overwrite().save(save_path )
        print(f"\n MEILLEUR MODÈLE : {best_model_name} sauvegardé dans '{save_path}' avec un RMSE de {best_rmse:.2f}")
    
    return best_model_fit

In [36]:
training_evaluation_model(df_silver)


Entraînement du modèle : Linear Regression...


Modèle: Linear Regression
R²: 0.4670 | MAE: 93.33 | RMSE: 125.14
Entraînement du modèle : Linear Regression ElasticNet...
Modèle: Linear Regression ElasticNet
R²: 0.4680 | MAE: 93.30 | RMSE: 125.02
Entraînement du modèle : Random Forest...


Modèle: Random Forest
R²: 0.3864 | MAE: 106.71 | RMSE: 134.26


26/01/23 11:59:29 ERROR FileOutputCommitter: Mkdirs failed to create file:/mnt/c/Users/khadija/Desktop/BTC-Prediction/quantai-btc-price-prediction-platform/ml/nootbooks/ml/saved_model/Linear Regression ElasticNet_pipeline/metadata/_temporary/0
26/01/23 11:59:29 ERROR Executor: Exception in task 0.0 in stage 1936.0 (TID 1772)
java.io.IOException: Mkdirs failed to create file:/mnt/c/Users/khadija/Desktop/BTC-Prediction/quantai-btc-price-prediction-platform/ml/nootbooks/ml/saved_model/Linear Regression ElasticNet_pipeline/metadata/_temporary/0/_temporary/attempt_202601231159298495287223769551307_4053_m_000000_0 (exists=false, cwd=file:/mnt/c/Users/khadija/Desktop/BTC-Prediction/quantai-btc-price-prediction-platform/ml/nootbooks)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.cre

Py4JJavaError: An error occurred while calling o7529.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:106)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1091)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:410)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1089)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopFile$4(PairRDDFunctions.scala:1062)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOper

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler ,StandardScaler
from pyspark.ml.regression import FMRegressor, LinearRegression


def training_evaluation_model(data):
 
    features_cols = ["returns", "MA_5", "MA_10", "taker_ratio"]
    
    target = "close_t_plus_10"

    assembler = VectorAssembler(inputCols=features_cols, outputCol="features",handleInvalid="skip")
    window = Window.orderBy("open_time_ts")
    df = data.withColumn("time_index", F.row_number().over(window))

   
    # Séparation séquentielle
    total_count = df.count()
    split_point = int(0.8 * total_count)


    
    train_data = df.filter(F.col("time_index") <= split_point)
    test_data= df.filter(F.col("time_index") > split_point)
    

    
    
    # Transormations
    
    scaler = StandardScaler(inputCol="features", outputCol="features_scaled", withStd=True, withMean=True)

    models = [
        
        # Modèle 2: Linear Regression avec ElasticNet (L1 + L2)
        # regParam=0.1 (force), elasticNetParam=0.5 (mélange L1/L2)
        ("Linear Regression ElasticNet", LinearRegression(featuresCol="features_scaled", labelCol=target, 
                                                           regParam=0.1, elasticNetParam=0.5)),
        
        # On garde le GBT pour comparer
        ("GBT Regressor", GBTRegressor(featuresCol="features_scaled", labelCol=target, maxIter=30))
    ]
    #  Évaluateurs
    evaluator_r2 = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="r2")
    evaluator_mae = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="mae")
    evaluator_rmse = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
    
    best_rmse = float("inf")
    best_model_name = None
    for model_name, model_algo in models:
        print(f"Entraînement du modèle : {model_name}...")
    
        # Créer pipeline
        pipeline = Pipeline(stages=[assembler, scaler, model_algo])

        # Entraîner modèle
        model_fit = pipeline.fit(train_data)
        # Prédiction
        preds = model_fit.transform(test_data)
        
        # Évaluation
        r2 = evaluator_r2.evaluate(preds)
        mae = evaluator_mae.evaluate(preds)
        rmse = evaluator_rmse.evaluate(preds)
        
        print(f"Modèle: {model_name}")

        print(f"R²: {r2:.4f} | MAE: {mae:.2f} | RMSE: {rmse:.2f}")
        
        if rmse < best_rmse:
            best_rmse = rmse
            best_model_fit = model_fit
            best_model_name = model_name


    if best_model_fit :
        save_path  = f"saved_model/{best_model_name}_pipeline"
        model_fit.write().overwrite().save(save_path )
        print(f"\n MEILLEUR MODÈLE : {best_model_name} sauvegardé dans '{save_path}' avec un RMSE de {best_rmse:.2f}")
    
    return best_model_fit
training_evaluation_model(df_silver)


Entraînement du modèle : Linear Regression ElasticNet...


Modèle: Linear Regression ElasticNet
R²: 0.4680 | MAE: 93.30 | RMSE: 125.02
Entraînement du modèle : Huber Regressor...


Modèle: Huber Regressor
R²: -0.6222 | MAE: 161.58 | RMSE: 218.30
Entraînement du modèle : GBT Regressor...


Modèle: GBT Regressor
R²: 0.3227 | MAE: 111.40 | RMSE: 141.06



 MEILLEUR MODÈLE : Linear Regression ElasticNet sauvegardé dans 'saved_model/Linear Regression ElasticNet_pipeline' avec un RMSE de 125.02


PipelineModel_fdce109b8011